In [1]:
import os
import numpy as np
import pandas as pd
from numba import jit
from tqdm import tqdm
import statsmodels.api as sm
import sys
sys.path.append('D:\实习\研报复现\QuantDev\ymtool')
import Alpha as alpha

import warnings
warnings.filterwarnings('ignore')

# UTD

In [4]:
freeshare = alpha.load_data('pv\\stock_shareout')

In [15]:
data = os.listdir(r"D:\实习\研报复现\dataAll\stockMinData\close")

res = []
for date in tqdm(data):
    volume_path = os.path.join(r"D:\实习\研报复现\dataAll\stockMinData\volume",date)
    date = pd.to_datetime(date.split('.')[0],format='%Y%m%d')
    volume = pd.read_parquet(volume_path)
    TurnVol_daily = np.divide(volume,freeshare.loc[date]).std(axis=0)
    TurnVol_daily.name = date
    res.append(TurnVol_daily)

TurnVol_daily = pd.concat(TurnVol_daily,axis=1).T
UTD = (TurnVol_daily.rolling(20,min_periods=1).std().div(TurnVol_daily.rolling(20,min_periods=1).mean(),axis=0)).iloc[19:]

In [20]:
UTDNeu = alpha.mvind_neutral(UTD)

100%|██████████████████████████████████████████████████████████████████████████████| 2473/2473 [00:39<00:00, 62.54it/s]


In [21]:
alpha.to_fctDataBase(UTDNeu.dropna(axis=0,how='all'),'UTDNeu')